In [ ]:
import pandas as pd
import numpy as np
import pylab as plt

import mysql.connector

mydb = mysql.connector.connect(
  host="172.28.200.50",
  user="root",
  port=3306,
  passwd="insert-password-here",
  database="iiwari_org")


mycursor = mydb.cursor(dictionary=True)
mycursor.execute("SELECT * FROM CleanSensorData;" );

df = pd.DataFrame(mycursor.fetchall())
df.head()

# Plot miltä näyttää x ja y koordinaatit

In [ ]:
img = plt.imread("whitepic.png")
fig, ax = plt.subplots()
ax.imshow(img,extent=(-5000,15000,10000,-5000),aspect='auto')
plt.rcParams['figure.figsize'] = [20,10]
plt.plot(df.x, df.y, 'o')

# Zscorella suurimmat hypyt pois

In [ ]:
from scipy.stats import zscore
from scipy import stats

z_scores = stats.zscore(df)
abs_z_scores = np.abs(z_scores)
filtered_entries = (abs_z_scores < 4).all(axis=1)
new_df = df[filtered_entries]


# Laitetaan timestamp datetime muotoon ja korjataan timezonet

In [ ]:
# Leikataan älämölöt pois
new_df['timestamp'] = new_df['timestamp'].astype(str)
new_df['timestamp'] = new_df['timestamp'].str.slice(2, -7)
new_df['timestamp'] = new_df['timestamp'].astype('datetime64[ns]')

 # Pyöristys millisekuntien tarkkuudelle
new_df.timestamp = new_df.timestamp.dt.round("ms")

new_df.timestamp = new_df.timestamp.dt.tz_localize('UTC')

    # Muunnetaan Suomen aikaan. Tämä huomioi kesä- ja talviajan.
new_df.timestamp = new_df.timestamp.dt.tz_convert('Europe/Helsinki')

new_df

# Poistetaan dublikaatit node_id ja timestampin mukaan

In [ ]:
new_df = new_df.drop_duplicates(subset=['node_id','timestamp'], keep='first')

# Node_id:t järkevään muotoon

In [ ]:
new_df['node_id'] = pd.factorize(new_df['node_id'])[0] + 1
new_df

# Tallennetaan uusi dataframe .csv tiedostoon

In [ ]:
new_df.to_csv('./clean_data.csv',index=False)